In [ ]:
import nengo
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import nengo_deeplearning.backends.theano.optimizers as opt
from nengo_deeplearning.backends.theano.networks import RNN
from nengo_deeplearning.backends.theano.layers import Generic, GatedRecurrent, Dense
from nengo_deeplearning.inputs.mnist import load_mnist

In [ ]:
trX, teX, trY, teY = load_mnist(data_dir='.')  # Will download to data_dir if they don't exist

In [ ]:
# RNN processes a size 28 vector at a time scanning from left to right 
layers = [
    Generic(size=28),
    GatedRecurrent(size=512, p_drop=0.2),
    Dense(size=10, activation='softmax', p_drop=0.5)
]

# A bit of l2 helps with generalization, higher momentum helps convergence
optimizer = opt.NAG(momentum=0.95, regularizer=opt.Regularizer(l2=1e-4))

# Linear iterator for real valued data, cce cost for softmax
model = RNN(layers=layers, optimizer=optimizer, iterator='linear', cost='mse')
model.fit(trX, trY, n_epochs=10)

tr_preds = model.predict(trX[:len(teY)])
te_preds = model.predict(teX)

tr_acc = np.mean(np.argmax(trY[:len(teY)], axis=1) == np.argmax(tr_preds, axis=1))
te_acc = np.mean(np.argmax(teY, axis=1) == np.argmax(te_preds, axis=1))

print "  ====== Results ======"
print "Train accuracy %s\tTest accuracy %s" % (tr_acc, te_acc)